In [8]:
import numpy as np
print("reading data files...",end="")
train_text = []
train_topic = []
test_text = []
test_topic = []
filename = "HAM-Train-Test\HAM-Train.txt"
file = open(filename,"r",encoding='utf-8')
for line in file.readlines():
    i = line.find("@")
    train_topic.append(line[:i])
    train_text.append(line[i+10:].split(" ")[:])
filename = "HAM-Train-Test\HAM-Test.txt"
file = open(filename,"r",encoding='utf-8')
for line in file.readlines():
    i = line.find("@")
    test_topic.append(line[:i])
    test_text.append(line[i+10:].split(" ")[:])
    
classes = set(train_topic)

words = set()
for text in train_text:
    for word in text:
        words.add(word)
print("done")

reading data files...done


In [9]:
# calculating unigram and bigram model
print("calculating Unigram and Bigram Models...",end="")
uni_dic = {}
bi_dic = {}
classes_prob = {}
num_words_byTopic= {}
for t in classes:
    uni_dic[t]={}
    bi_dic[t]={}
    classes_prob[t]=0
    num_words_byTopic[t]=0
    
for c in train_topic:
    classes_prob[c]+=1
x = 0
for t in classes:
    classes_prob[t]=classes_prob[t]/len(train_topic)
    total = 0
    for i in range(len(train_text)):
        if train_topic[i]==t:
            total+=len(train_text)
    num_words_byTopic[t] = total
    x+=1
num=0
for i in range(len(train_text)):
    data = train_text[i]
    for j in range(len(data)-1):
        uni_dic[train_topic[i]][data[j]]=0
        bi_dic[train_topic[i]][data[j],data[j+1]]=0
    uni_dic[train_topic[i]][data[j+1]]=0
    num+=1
    if num == 3000:
        print(".",end="")
        num=0
for i in range(len(train_text)):
    data = train_text[i]
    for j in range(len(data)-1):
        uni_dic[train_topic[i]][data[j]]+=1
        bi_dic[train_topic[i]][data[j],data[j+1]]+=1
    uni_dic[train_topic[i]][data[j+1]]+=1
    num+=1
    if num == 3000:
        print(".",end="")
        num=0
print("done")

calculating Unigram and Bigram Models........done


In [18]:
# calculating test predictions using unigram and bigram
uni_pred = []
bi_pred = []
lambda1 = 0.2
lambda2 = 0.8
print("classifying test set...",end="")
test_num = 0
for i in range(len(test_text)):
    test = test_text[i]
    this_test_prob_uni = {}
    this_test_prob_bi = {}
    for x in classes:
        this_test_prob_uni[x] = 0 
        this_test_prob_bi[x] = 0 
    for j in range(len(test)-1):
        word = test[j]
        word2 = (test[j],test[j+1])
        for topic in classes:
            topic_prob_uni = 0
            topic_prob_bi = 0
            
            prob1 = uni_dic[topic].get(word)
            if prob1 == None:
                prob1 = 0.000001
            prob1 = np.log(prob1/num_words_byTopic[topic])
            
            prob2 = bi_dic[topic].get(word2)
            prob22 = uni_dic[topic].get(word2[1])
            prob21 = uni_dic[topic].get(word2[0])
            if prob2 == None:
                prob2 = 0.000000001
                if prob22 == None:
                    prob22 = 0.000001
                prob2 = lambda1*(prob2) + lambda2*(prob22/num_words_byTopic[topic])
            else:
                prob2 = prob2/prob21        
            prob2 = np.log(prob2)
            this_test_prob_uni[topic] += prob1
            this_test_prob_bi[topic] += prob2
    this_test_prob_uni[topic] += np.log(classes_prob[topic])
    this_test_prob_bi[topic] += np.log(classes_prob[topic])
    uni_pred.append(this_test_prob_uni.copy())
    bi_pred.append(this_test_prob_bi.copy())   
    test_num+=1
    if test_num == 100:
        print(".",end="")
        test_num=0
print("done")
index = 0
hit = 0
topic = []
for i in classes:
    topic.append(i)
for x in uni_pred:
    temp = []
    for t in topic:
        temp.append(x[t])
    if topic[np.argmax(temp)]==test_topic[index]:
        hit += 1
    index += 1

hit2 = 0
index = 0
for x in bi_pred:
    temp = []
    for t in topic:
        temp.append(x[t])
    if topic[np.argmax(temp)]==test_topic[index]:
        hit2 += 1
    index += 1

classifying test set...........done


In [16]:
#calculating percition and recall
topic = []
for i in classes:
    topic.append(i)

topic_test = []
for t in test_topic:
    topic_test.append(topic.index(t))
pred_uni = []
for pred in uni_pred:
    temp = []
    for t in topic:
        temp.append(pred[t])
    pred_uni.append(np.argmax(temp))
pred_bi = []
for pred in bi_pred:
    temp = []
    for t in topic:
        temp.append(pred[t])
    pred_bi.append(np.argmax(temp))

In [17]:
uni_result_MAT = []
for i in range(len(topic)):
    uni_result_MAT.append([])
    for j in range(len(topic)):
        uni_result_MAT[i].append(0)
for i in range(len(pred_uni)):
    uni_result_MAT[topic_test[i]][pred_uni[i]]+=1

bi_result_MAT = []
for i in range(len(topic)):
    bi_result_MAT.append([])
    for j in range(len(topic)):
        bi_result_MAT[i].append(0)
for i in range(len(pred_uni)):
    bi_result_MAT[topic_test[i]][pred_bi[i]]+=1

precision_uni = []
precision_bi = []
recall_uni = []
recall_bi = []
#percision
for t in range(len(topic)):
    tu = uni_result_MAT[t][t]
    fu = 0
    tb = bi_result_MAT[t][t]
    fb = 0
    for pu in uni_result_MAT:
        fu+=pu[t]
    for pb in bi_result_MAT:
        fb+=pb[t]
    precision_uni.append(tu/fu)
    precision_bi.append(tb/fb)
#recall
for t in range(len(topic)):
    tu = uni_result_MAT[t][t]
    fu = sum(uni_result_MAT[t])
    tb = bi_result_MAT[t][t]
    fb = sum(bi_result_MAT[t])
    recall_uni.append(tu/fu)
    recall_bi.append(tb/fb)
f1_uni = []
f1_bi = []
for i in range(len(topic)):
    f1_uni.append((2*precision_uni[i]*recall_uni[i])/(precision_uni[i]+recall_uni[i]))
    f1_bi.append((2*precision_bi[i]*recall_bi[i])/(precision_bi[i]+recall_bi[i]))
print("RESULTS:")
print("______________________________")
print("unigram true predictions: "+str(hit)+"\\"+str(len(test_text)))
print("precision(UNIgram):")
for i in range(len(topic)):
    print("   "+topic[i]+" : "+str(precision_uni[i])[:5])
print("recall(UNIgram):")
for i in range(len(topic)):
    print("   "+topic[i]+" : "+str(recall_uni[i])[:5])
print("F1_score(UNIgram):")
for i in range(len(topic)):
    print("   "+topic[i]+" : "+str(f1_uni[i])[:5])
print("______________________________")
print("bigram true predictions: "+str(hit2)+"\\"+str(len(test_text)))
print("precision(BIgram):")
for i in range(len(topic)):
    print("   "+topic[i]+" : "+str(precision_bi[i])[:5])
print("recall(BIgram):")
for i in range(len(topic)):
    print("   "+topic[i]+" : "+str(recall_bi[i])[:5])
print("F1_score(BIgram):")
for i in range(len(topic)):
    print("   "+topic[i]+" : "+str(f1_bi[i])[:5])

RESULTS:
______________________________
unigram true predictions: 693\860
precision(UNIgram):
   سیاسی : 0.895
   اقتصاد : 0.978
   اجتماعی : 0.531
   ورزش : 0.995
   ادب و هنر : 0.797
recall(UNIgram):
   سیاسی : 0.685
   اقتصاد : 0.658
   اجتماعی : 0.926
   ورزش : 0.925
   ادب و هنر : 0.948
F1_score(UNIgram):
   سیاسی : 0.776
   اقتصاد : 0.787
   اجتماعی : 0.675
   ورزش : 0.959
   ادب و هنر : 0.866
______________________________
bigram true predictions: 673\860
precision(BIgram):
   سیاسی : 0.916
   اقتصاد : 0.883
   اجتماعی : 0.926
   ورزش : 0.976
   ادب و هنر : 0.281
recall(BIgram):
   سیاسی : 0.77
   اقتصاد : 0.900
   اجتماعی : 0.386
   ورزش : 0.929
   ادب و هنر : 0.931
F1_score(BIgram):
   سیاسی : 0.836
   اقتصاد : 0.892
   اجتماعی : 0.545
   ورزش : 0.952
   ادب و هنر : 0.432


In [40]:
print(f1_uni)

[0.8682170542635659, 0.7909604519774013, 0.7740112994350283, 0.9612756264236901, 0.6801801801801802]


In [16]:

    test = test_text[i]
    this_test_prob_uni = []
    this_test_prob_bi = []
    for topic in classes:
        this_topic_prob_uni = 0
        this_topic_prob_bi = 0
        for j in range(len(test)-1):
            word = test[j]
            word2 = (test[j],test[j+1])
            prob = uni_dic[topic].get(word)
            prob2 = bi_dic[topic].get(word2)
            if prob == None:
                prob = 0.000001
            if prob2 == None:
                prob2 = 0.000001
            prob = prob/num_words_byTopic[topic]
            numenator = test.count(word2[0]) + 1
            param1 = uni_dic[topic].get(word2[1])
            if param1 == None:
                param1 = 0.000001
            param2 = (prob2/numenator)
            prob2 = lambda1*param1 + lambda2*param2
            this_topic_prob_uni+=np.log(prob)
            this_topic_prob_bi+=np.log(prob2)
        this_topic_prob_uni+=np.log(classes_prob[topic])
        this_topic_prob_bi+=np.log(classes_prob[topic])
        this_test_prob_uni.append([topic,this_topic_prob_uni])
        this_test_prob_bi.append([topic,this_topic_prob_bi])
    uni_pred.append(this_test_prob_uni)
    bi_pred.append(this_test_prob_bi)
z=0
cp = 0
for x in uni_pred:
    temp = []
    for p in x:
        temp.append(p[1])
    if (test_topic[z]==x[np.argmax(temp)][0]) :
        cp+=1
    z+=1
cp2 = 0
z=0
print(cp,len(uni_pred))
for x in bi_pred:
    temp = []
    for p in x:
        temp.append(p[1])
    if (test_topic[z]==x[np.argmax(temp)][0]) :
        cp2+=1
    z+=1
print(cp2,len(bi_pred))

KeyError: 2